In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.linear_model as sk
import sklearn.svm as sv
from glove_routines import *
from text_classifier import *
%load_ext autoreload
%autoreload 2

In [10]:
def predict_labels(flag=".npy"):
    #Load the training set
    path_neg = str("data/trainingset_neg"+flag)
    path_pos = str("data/trainingset_pos"+flag)
    ts_neg = np.load(path_neg)
    ts_pos = np.load(path_pos)    
    #Train a Linear Classifier: Train a linear classifier (e.g. logistic regression or SVM) on your constructed 
    #features, using the scikit learn library, or your own code from the earlier labs. Recall that the labels 
    #indicate if a tweet used to contain a :) or :( smiley.
    training_set = np.concatenate((ts_neg,ts_pos))
    y = training_set[:,0]
    X = training_set[:,1:np.shape(training_set)[1]]
    #Now we load and predict the data
        
    #Construct the logistic regressor
    LRR = sk.LogisticRegressionCV()
    LR = sk.LogisticRegression()
    #LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, 
    #class_weight=None, random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, 
    #warm_start=False, n_jobs=1)[source]¶
    #http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
    #train the logistic regressor
    errorR = []
    errorN = []
    error_selfR = []
    error_selfnR = []
    kf = ms.KFold(n_splits=10,shuffle=True)
    for train_idx, test_idx in kf.split(X):
        train_set = X[train_idx]
        test_set = X[test_idx]
        train_target = y[train_idx]
        test_target = y[test_idx]    
        LRR.fit(train_set,train_target)
        LR.fit(train_set,train_target)
        predictions_temp = LRR.predict(test_set)
        predictions_self_R = LRR.predict(train_set)
        predictions_ntemp = LR.predict(test_set)   
        predictions_self_nR = LR.predict(train_set)
        error = np.sum(np.power(predictions_temp-test_target,2))/np.shape(predictions_temp)[0]
        error_self_R = np.sum(np.power(predictions_self_R-train_target,2))/np.shape(predictions_self_R)[0]
        error_self_nR = np.sum(np.power(predictions_self_nR-train_target,2))/np.shape(predictions_self_nR)[0]
        errorn = np.sum(np.power(predictions_ntemp-test_target,2))/np.shape(predictions_ntemp)[0]
        errorR.append(error)
        error_selfR.append(error_self_R)
        error_selfnR.append(error_self_nR)
        errorN.append(errorn)
        print("Yet, error is",error)
        print("Errorn is",errorn)
        print("Error_selfR is",error_self_R)
        print("Error_selfnR is",error_self_nR)


    return errorR, errorN, error_selfR, error_selfnR
        
    
errorR, errornR, error_selfR,error_selfnR = predict_labels()
    

Yet, error is 0.3641
Errorn is 0.36455
Error_selfR is 0.362766666667
Error_selfnR is 0.362755555556
Yet, error is 0.36295
Errorn is 0.363
Error_selfR is 0.362966666667
Error_selfnR is 0.362855555556
Yet, error is 0.3643
Errorn is 0.3644
Error_selfR is 0.363111111111
Error_selfnR is 0.363155555556
Yet, error is 0.365
Errorn is 0.3654
Error_selfR is 0.362794444444
Error_selfnR is 0.36275
Yet, error is 0.3611
Errorn is 0.36095
Error_selfR is 0.363811111111
Error_selfnR is 0.3638
Yet, error is 0.36785
Errorn is 0.3678
Error_selfR is 0.362705555556
Error_selfnR is 0.362711111111
Yet, error is 0.36215
Errorn is 0.36185
Error_selfR is 0.362888888889
Error_selfnR is 0.363
Yet, error is 0.3626
Errorn is 0.36285
Error_selfR is 0.363111111111
Error_selfnR is 0.363083333333
Yet, error is 0.3617
Errorn is 0.36085
Error_selfR is 0.363072222222
Error_selfnR is 0.36295
Yet, error is 0.361
Errorn is 0.36115
Error_selfR is 0.363038888889
Error_selfnR is 0.362994444444


In [12]:
print(errornR)
print(errorR)
print(error_selfR)
print(error_selfnR)

er_R = sum(errorR)/np.shape(errorR)[0]
er_nR = sum(errornR)/np.shape(errornR)[0]
er_s_R = sum(error_selfR)/np.shape(error_selfR)[0]
er_s_nR = sum(error_selfnR)/np.shape(error_selfnR)[0]
print(er_R)
print(er_nR)
print(er_s_R)
print(er_s_nR)

[0.36454999999999999, 0.36299999999999999, 0.3644, 0.3654, 0.36094999999999999, 0.36780000000000002, 0.36185, 0.36285000000000001, 0.36085, 0.36115000000000003]
[0.36409999999999998, 0.36294999999999999, 0.36430000000000001, 0.36499999999999999, 0.36109999999999998, 0.36785000000000001, 0.36215000000000003, 0.36259999999999998, 0.36170000000000002, 0.36099999999999999]
[0.36276666666666668, 0.36296666666666666, 0.36311111111111111, 0.36279444444444442, 0.36381111111111109, 0.36270555555555556, 0.36288888888888887, 0.36311111111111111, 0.36307222222222224, 0.36303888888888891]
[0.36275555555555555, 0.36285555555555554, 0.36315555555555556, 0.36275000000000002, 0.36380000000000001, 0.3627111111111111, 0.36299999999999999, 0.36308333333333331, 0.36294999999999999, 0.36299444444444445]
0.363275
0.36328
0.363026666667
0.363005555556


In [ ]:
def predict_labels(flag=".npy"):
    #Load the training set
    path_neg = str("data/trainingset_neg"+flag)
    path_pos = str("data/trainingset_pos"+flag)
    ts_neg = np.load(path_neg)
    ts_pos = np.load(path_pos)    
    #Train a Linear Classifier: Train a linear classifier (e.g. logistic regression or SVM) on your constructed 
    #features, using the scikit learn library, or your own code from the earlier labs. Recall that the labels 
    #indicate if a tweet used to contain a :) or :( smiley.
    training_set = np.concatenate((ts_neg,ts_pos))
    y = training_set[:,0]
    X = training_set[:,1:np.shape(training_set)[1]]
    #Now we load and predict the data
        
    #Construct the logistic regressor
    LRR = sk.LogisticRegressionCV()
    LR = sv.SVC()
    #LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, 
    #class_weight=None, random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, 
    #warm_start=False, n_jobs=1)[source]¶
    #http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
    #train the logistic regressor
    errorR = []
    errorN = []
    error_selfR = []
    error_selfnR = []
    kf = ms.KFold(n_splits=2,shuffle=True)
    for train_idx, test_idx in kf.split(X):
        train_set = X[train_idx]
        test_set = X[test_idx]
        train_target = y[train_idx]
        test_target = y[test_idx]    
        LRR.fit(train_set,train_target)
        LR.fit(train_set,train_target)
        predictions_temp = LRR.predict(test_set)
        predictions_self_R = LRR.predict(train_set)
        predictions_ntemp = LR.predict(test_set)   
        predictions_self_nR = LR.predict(train_set)
        error = np.sum(np.power(predictions_temp-test_target,2))/np.shape(predictions_temp)[0]
        error_self_R = np.sum(np.power(predictions_self_R-train_target,2))/np.shape(predictions_self_R)[0]
        error_self_nR = np.sum(np.power(predictions_self_nR-train_target,2))/np.shape(predictions_self_nR)[0]
        errorn = np.sum(np.power(predictions_ntemp-test_target,2))/np.shape(predictions_ntemp)[0]
        errorR.append(error)
        error_selfR.append(error_self_R)
        error_selfnR.append(error_self_nR)
        errorN.append(errorn)
        print("Yet, error is",error)
        print("Errorn is",errorn)
        print("Error_selfR is",error_self_R)
        print("Error_selfnR is",error_self_nR)


    return errorR, errorN, error_selfR, error_selfnR
        
    
errorRs, errornRs, error_selfRs,error_selfnRs = predict_labels()
    